<a href="https://colab.research.google.com/github/geervani26/-Facial-Emotion-Recognition-using-OpenCV/blob/main/AIExpenseTracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# SpendGenie full app - Google Colab single
# Paste this whole cell into Colab and run.

# Install dependencies
!pip install -q flask flask_sqlalchemy flask_bcrypt pyngrok pandas plotly

import os, io, uuid, shutil
from pathlib import Path
from datetime import datetime
from flask import Flask, render_template, render_template_string, request, redirect, session, jsonify, send_file, url_for
from flask_sqlalchemy import SQLAlchemy
from flask_bcrypt import Bcrypt
from pyngrok import ngrok
import pandas as pd
import numpy as np

# ----------------- Configuration -----------------
NGROK_TOKEN = "35KueGADcf95Z3RT8YYZ5KVaWTl_NbbBqKZ8wzPkNaN27Hvi"  # selected token A
# Developer note: using user-uploaded dataset path as default fallback URL
DEFAULT_DATA_PATH = "/mnt/data/Personal_Finance_Dataset.csv"  # existing uploaded file path
BASE_DIR = Path("/tmp/spendgenie_app")
UPLOAD_ROOT = BASE_DIR / "uploads"
TEMPLATE_DIR = BASE_DIR / "templates"
STATIC_DIR = BASE_DIR / "static"
for d in (BASE_DIR, UPLOAD_ROOT, TEMPLATE_DIR, STATIC_DIR):
    d.mkdir(parents=True, exist_ok=True)

# ----------------- Flask app -----------------
app = Flask(__name__, template_folder=str(TEMPLATE_DIR), static_folder=str(STATIC_DIR))
app.secret_key = "spendgenie-secret-key-2025"
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///" + str(BASE_DIR / "users.db")
app.config["SQLALCHEMY_TRACK_MODIFICATIONS"] = False

db = SQLAlchemy(app)
bcrypt = Bcrypt(app)

# ----------------- Database model -----------------
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80))
    email = db.Column(db.String(120), unique=True)
    password = db.Column(db.String(200))

with app.app_context():
    db.create_all()

# ----------------- Utilities -----------------
def safe_session_id():
    sid = session.get("session_id")
    if not sid:
        sid = str(uuid.uuid4())
        session["session_id"] = sid
    return sid

def user_upload_dir():
    sid = safe_session_id()
    d = UPLOAD_ROOT / sid
    d.mkdir(parents=True, exist_ok=True)
    return d

def save_uploaded_csv(file_storage):
    updir = user_upload_dir()
    filename = "uploaded.csv"
    path = updir / filename
    file_storage.save(str(path))
    return str(path)

def get_user_csv_path():
    updir = user_upload_dir()
    path = updir / "uploaded.csv"
    if path.exists():
        return str(path)
    # fallback: copy global default dataset into user folder if exists
    if os.path.exists(DEFAULT_DATA_PATH):
        try:
            shutil.copy(DEFAULT_DATA_PATH, str(path))
            return str(path)
        except Exception:
            return None
    return None

def guess_col(candidates, columns):
    cols_lower = {c.lower(): c for c in columns}
    for name in candidates:
        if name.lower() in cols_lower:
            return cols_lower[name.lower()]
    for name in candidates:
        for c in columns:
            if name.lower() in c.lower():
                return c
    return None

def clean_amount_val(x):
    try:
        if pd.isna(x):
            return np.nan
        s = str(x).strip()
        if s == "":
            return np.nan
        if s.startswith("(") and s.endswith(")"):
            s = "-" + s[1:-1]
        s = s.replace(",", "")
        s2 = "".join(ch for ch in s if ch.isdigit() or ch in ".-+")
        if s2 in ("", ".", "+", "-", "+.", "-."):
            return np.nan
        return float(s2)
    except:
        return np.nan

def load_and_prepare(csv_path):
    df = pd.read_csv(csv_path, low_memory=False)
    df.columns = [c.strip() for c in df.columns]
    cols = df.columns.tolist()
    date_col = guess_col(["date","transaction_date","txn_date","posted_date","timestamp"], cols)
    amt_col = guess_col(["amount","amt","value","transaction_amount","debit","credit"], cols)
    cat_col = guess_col(["category","cat","expense_category","merchant_category","type","merchant"], cols)
    desc_col = guess_col(["description","memo","narration","details"], cols)
    if not date_col or not amt_col:
        raise ValueError("Could not detect date and amount columns. Ensure CSV has date & amount columns.")
    df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
    df = df.dropna(subset=[date_col])
    df["__raw_amount__"] = df[amt_col]
    df["amount_clean"] = df[amt_col].map(clean_amount_val)
    df = df.dropna(subset=["amount_clean"])
    if cat_col:
        df["category_clean"] = df[cat_col].fillna("Uncategorized").astype(str)
    else:
        df["category_clean"] = "Uncategorized"
    if desc_col:
        df["description_clean"] = df[desc_col].astype(str)
    else:
        df["description_clean"] = ""
    df["year"] = df[date_col].dt.year
    df["month"] = df[date_col].dt.month
    df["month_year"] = df[date_col].dt.to_period("M").astype(str)
    df["day"] = df[date_col].dt.date
    nonzero = df[df["amount_clean"] != 0]["amount_clean"]
    neg_frac = (nonzero < 0).mean() if len(nonzero) > 0 else 0
    if neg_frac >= 0.5:
        df["expense"] = df["amount_clean"].apply(lambda v: -v if v < 0 else 0.0)
        df["income"] = df["amount_clean"].apply(lambda v: v if v > 0 else 0.0)
        sign_rule = "expenses_negative"
    else:
        type_col = guess_col(["type","transaction_type","kind"], cols)
        if type_col and type_col in df.columns:
            t = df[type_col].astype(str).str.lower()
            exp_mask = t.isin(["expense","debit","withdrawal","payment","spent"])
            df["expense"] = df["amount_clean"].where(exp_mask, 0).abs()
            df["income"] = df["amount_clean"].where(~exp_mask, 0).abs()
            sign_rule = "type_based"
        else:
            df["expense"] = df["amount_clean"].apply(lambda v: v if v > 0 else 0.0)
            df["income"] = df["amount_clean"].apply(lambda v: -v if v < 0 else 0.0)
            sign_rule = "positive_expense"
    return df, {"date_col": date_col, "amt_col": amt_col, "cat_col": cat_col, "sign_rule": sign_rule}

# ----------------- Templates & static files -----------------
# CyberDark + Light CSS (keeps login/upload/dashboard unchanged)
style_css = """
/* CyberDark + light toggle */
body{margin:0;font-family:Inter,Arial;background:#06060b;color:#e6f3ff}
body.light{background:#f6f8ff;color:#081129}
header{display:flex;justify-content:space-between;align-items:center;padding:18px 30px;background:linear-gradient(90deg,#041224,#063b66)}
header .title{font-size:20px;color:#9ee2ff;font-weight:700}
.header-right{display:flex;gap:10px;align-items:center}
a.link{color:#cfe9ff;text-decoration:none;padding:8px 10px;border-radius:8px}
.main-container{max-width:1160px;margin:18px auto;padding:0 18px}
.filters{display:flex;gap:12px;flex-wrap:wrap;align-items:center;justify-content:center;margin-bottom:12px;padding:12px;background:rgba(255,255,255,0.02);border-radius:10px}
.filters select,.filters input{padding:10px;border-radius:8px;border:none;background:rgba(255,255,255,0.02);color:inherit}
button{background:#0ea5ff;color:white;border:none;padding:10px 12px;border-radius:8px;cursor:pointer}
button:hover{opacity:0.95}
.ai-summary{background:rgba(255,255,255,0.02);padding:14px;border-radius:12px;margin:10px 0;text-align:center}
.grid{display:grid;grid-template-columns:repeat(2,1fr);gap:18px}
.card{background:linear-gradient(180deg,rgba(255,255,255,0.02),rgba(255,255,255,0.01));padding:14px;border-radius:12px;min-height:220px}
.empty{color:#93c5fd;text-align:center;padding:30px}
.auth-box{width:360px;margin:60px auto;padding:30px;background:rgba(255,255,255,0.02);border-radius:14px;text-align:center}
.auth-box input{width:100%;padding:12px;margin:8px 0;border-radius:8px;border:none;background:rgba(255,255,255,0.01);color:inherit}
.auth-box button{width:100%;padding:12px;border-radius:8px;border:none;background:#0ea5ff;color:white}
.preview{width:100%;border-collapse:collapse}
.preview th,.preview td{padding:6px;border-bottom:1px solid rgba(255,255,255,0.03)}
.container-center{max-width:900px;margin:40px auto;padding:18px;text-align:center}
.error{color:#ffd166}
"""
(STATIC_DIR / "style.css").write_text(style_css)

# ----------------- Landing page (SpendGenie) -----------------
landing_html = """<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1" />
  <title>SpendGenie - AI Expense Forecasting</title>

  <style>
    @import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700;900&display=swap');

    :root {
      --button-radius: 8px;
    }

    * {
      margin: 0;
      padding: 0;
      box-sizing: border-box;
      font-family: 'Inter', sans-serif;
    }

    body {
      background: linear-gradient(to bottom, #ff4f8c, #ffffff);
      min-height: 100vh;
      display: flex;
      flex-direction: column;
      align-items: center;
      color: #333;
    }

    .navbar {
      width: 100%;
      padding: 25px 60px;
      display: flex;
      align-items: center;
      justify-content: space-between;
    }

    .logo {
      display: flex;
      align-items: center;
      gap: 10px;
      cursor: pointer;
    }

    .logo-circle {
      width: 40px;
      height: 40px;
      background: #fff;
      border-radius: 50%;
    }

    .menu {
      display: flex;
      gap: 35px;
      font-size: 18px;
    }

    .menu a {
      text-decoration: none;
      color: #fff;
      font-weight: 500;
    }

    .nav-buttons {
      display: flex;
      gap: 15px;
    }

    .btn-outline {
      padding: 8px 18px;
      border: 1.5px solid #fff;
      border-radius: var(--button-radius);
      background: transparent;
      color: #fff;
      cursor: pointer;
      transition: 0.2s;
    }

    .btn-outline:hover {
      background: rgba(255,255,255,0.2);
    }

    .btn-fill {
      padding: 8px 18px;
      border-radius: var(--button-radius);
      background: #5a2aff;
      color: #fff;
      cursor: pointer;
      border: none;
      transition: 0.2s;
      border-radius: var(--button-radius);
    }

    .hero {
      text-align: center;
      margin-top: 30px;
      max-width: 800px;
    }

    .hero h1 {
      font-size: 40px;
      font-weight: 800;
      color: #fff;
      line-height: 1.3;
    }

    .hero p {
      margin-top: 15px;
      font-size: 18px;
      color: #f5f5f5;
      line-height: 1.5;
    }

    .get-started {
      margin-top: 25px;
      padding: 12px 28px;
      font-size: 18px;
      border-radius: var(--button-radius);
      background: #5a2aff;
      border: none;
      color: #fff;
      cursor: pointer;
      transition: 0.2s;
    }

    .laptop-container {
      margin-top: 40px;
      width: 900px;
    }

    .laptop-container img {
      width: 100%;
      border-radius: 10px;
      box-shadow: 0px 10px 25px rgba(0,0,0,0.25);
    }

    footer {
      width: 100%;
      background: #222;
      color: #fff;
      padding: 30px 0;
      margin-top: 60px;
      text-align: center;
      font-size: 16px;
    }

    footer a {
      color: #ccc;
      margin: 0 12px;
      text-decoration: none;
      font-size: 15px;
    }

    footer a:hover {
      color: #fff;
    }
  </style>
</head>

<body>

  <!-- NAVBAR -->
  <div class="navbar">
    <div class="logo" onclick="location.href='/'">
      <div class="logo-circle"></div>
      <span style="font-size: 20px; color:#fff; font-weight:600;">SpendGenie</span>
    </div>

    <div class="menu">
      <a href="#">About</a>
      <a href="#">Contact Us</a>
    </div>

    <div class="nav-buttons">
      <button class="btn-outline" onclick="location.href='/login'">Sign In</button>
      <button class="btn-fill" onclick="location.href='/register'">Register</button>
    </div>
  </div>

  <!-- HERO CONTENT -->
  <div class="hero">
    <h1>Your Budget, Re-imagined With<br>Intelligence with SpendGenie</h1>

    <p>
      An AI that listens to the rhythm of your spending and predicts tomorrow’s trends.<br>
      Transform financial chaos into crystal-clear insights.
    </p>

    <button class="get-started" onclick="location.href='/upload'">Get Started</button>
  </div>

  <!-- MAIN DASHBOARD MOCKUP -->
  <div class="laptop-container">
    <img src="https://image2url.com/images/1763729644845-c1e18bd1-2559-4901-b201-9b4da762f269.png" alt="Dashboard Mockup">
  </div>

  <!-- FOOTER -->
  <footer>
    <p>© 2025 SpendGenie | All Rights Reserved</p>
    <div style="margin-top:12px;">
      <a href="#">About</a>
      <a href="#">Contact Us</a>
      <a href="#">Privacy Policy</a>
    </div>
  </footer>

</body>
</html>
"""

(TEMPLATE_DIR / "landing.html").write_text(landing_html)

# ----------------- KEEP Login / Register / Upload / Dashboard templates unchanged -----------------
# login.html
login_html = """<!doctype html>
<html><head><meta charset="utf-8"><title>Login - SpendGenie</title>
<link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
<div class="auth-box">
  <h2>SpendGenie</h2>
  <p>Sign in to continue</p>
  <form method="POST">
    <input name="email" type="email" placeholder="Email" required>
    <input name="password" type="password" placeholder="Password" required>
    <button type="submit">Login</button>
  </form>
  {% if error %}<p class="error">{{ error }}</p>{% endif %}
  <p>New? <a href="/register">Create account</a></p>
</div>
</body></html>"""
(TEMPLATE_DIR / "login.html").write_text(login_html)

# register.html
register_html = """<!doctype html>
<html><head><meta charset="utf-8"><title>Register - SpendGenie</title>
<link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
<div class="auth-box">
  <h2>Create account</h2>
  <form method="POST">
    <input name="username" placeholder="Username" required>
    <input name="email" type="email" placeholder="Email" required>
    <input name="password" type="password" placeholder="Password" required>
    <button type="submit">Register</button>
  </form>
  <p>Have an account? <a href="/login">Login</a></p>
</div>
</body></html>"""
(TEMPLATE_DIR / "register.html").write_text(register_html)

# upload.html (with preview) - unchanged behavior
upload_html = """<!doctype html>
<html><head><meta charset="utf-8"><title>Upload CSV - SpendGenie</title>
<link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}"></head>
<body>
<header><div style="padding:12px 20px"><a href="/logout" class="link">Logout</a> &nbsp; | &nbsp; <a href="/dashboard" class="link">Dashboard</a></div></header>
<div class="container-center">
  <h2>Upload your expense CSV</h2>
  <form method="POST" enctype="multipart/form-data">
    <input type="file" name="file" accept=".csv" required>
    <button type="submit">Upload & Preview</button>
  </form>

  {% if preview %}
    <h3>Preview (first 10 rows)</h3>
    {{ preview|safe }}
    <form method="POST" action="/load_dashboard">
      <button type="submit">Load & Continue to Dashboard</button>
    </form>
  {% endif %}
  {% if message %}
    <p class="error">{{ message }}</p>
  {% endif %}
</div>
</body></html>"""
(TEMPLATE_DIR / "upload.html").write_text(upload_html)

# dashboard.html (unchanged behavior; SpendGenie branding in header)
dashboard_html = """<!doctype html>
<html>
<head>
<meta charset="utf-8">
<title>SpendGenie Dashboard</title>
<script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
<link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
<header>
  <div class="title">⚡ SpendGenie</div>
  <div class="header-right">
    <button id="themeBtn">Toggle Theme</button>
    <a href="/upload" class="link">Upload</a>
    <a href="/logout" class="link">Logout</a>
  </div>
</header>

<main class="main-container">
  <section class="filters">
    <select id="year" onchange="applyFilters()"><option value=''>All Years</option></select>
    <select id="month" onchange="applyFilters()"><option value=''>All Months</option>
      <option value="1">Jan</option><option value="2">Feb</option><option value="3">Mar</option>
      <option value="4">Apr</option><option value="5">May</option><option value="6">Jun</option>
      <option value="7">Jul</option><option value="8">Aug</option><option value="9">Sep</option>
      <option value="10">Oct</option><option value="11">Nov</option><option value="12">Dec</option>
    </select>
    <input id="start" type="date" onchange="applyFilters()">
    <input id="end" type="date" onchange="applyFilters()">
    <button onclick="generateSummary()">AI Summary</button>
  </section>

  <section id="aiSummary" class="ai-summary">AI summary will appear here.</section>

  <section class="grid">
    <div class="card"><h3>Monthly Trends</h3><div id="monthly"></div></div>
    <div class="card"><h3>Category Breakdown</h3><div id="category"></div></div>
    <div class="card"><h3>Peak Spending Days</h3><div id="peak"></div></div>
    <div class="card"><h3>Yearly Comparison</h3><div id="yearly"></div></div>
  </section>
</main>

<script>
const themeBtn = document.getElementById("themeBtn");
themeBtn.addEventListener("click", ()=>{
  document.body.classList.toggle("light");
  localStorage.setItem("bw_theme", document.body.classList.contains("light") ? "light" : "dark");
});
if(localStorage.getItem("bw_theme") === "light") document.body.classList.add("light");

async function loadYears(){
  const res = await fetch("/years");
  const years = await res.json();
  const sel = document.getElementById("year");
  sel.innerHTML = "<option value=''>All Years</option>" + years.map(y=>`<option value=\"${y}\">${y}</option>`).join("");
}

async function applyFilters(){
  const filters = {
    year: document.getElementById("year").value || "",
    month: document.getElementById("month").value || "",
    start: document.getElementById("start").value || "",
    end: document.getElementById("end").value || ""
  };
  const params = new URLSearchParams(filters);
  const res = await fetch("/analysis?"+params.toString());
  const data = await res.json();
  updateCharts(data);
  updateAISummary(data.summary);
}

function updateCharts(data){
  Plotly.newPlot('monthly', [{
    x: data.monthly.map(r=>r.month_year),
    y: data.monthly.map(r=>r.amount),
    type: 'bar', marker:{color:'#0ea5ff'}
  }], {margin:{t:20}});

  Plotly.newPlot('category', [{
    labels: data.category.map(r=>r.category),
    values: data.category.map(r=>r.amount),
    type:'pie', textinfo:'label+percent'
  }], {margin:{t:20}});

  if (data.peak.length===1 && (data.peak[0].day==="No Data" || data.peak[0].date==="No Data")){
    document.getElementById('peak').innerHTML = '<div class="empty">No peak spending data available for selected filters.</div>';
  } else {
    const x = data.peak.map(r=>r.day || r.date);
    const y = data.peak.map(r=>r.amount);
    Plotly.newPlot('peak', [{x,y,mode:'lines+markers',line:{color:'#ff4d4d'}}], {margin:{t:20}});
  }

  Plotly.newPlot('yearly', [{
    x: data.yearly.map(r=>r.year),
    y: data.yearly.map(r=>r.amount),
    type:'scatter', mode:'lines+markers', line:{color:'#22bb88'}
  }], {margin:{t:20}});
}

function updateAISummary(summary){
  const el = document.getElementById('aiSummary');
  if (!summary) { el.innerText = "AI summary will appear here."; return; }
  el.innerText = `Total: ₹${Number(summary.total||0).toFixed(2)} · Top category: ${summary.top_cat || '-'} · Peak day: ${summary.peak_day || '-'}`;
}

async function generateSummary(){
  const filters = {
    year: document.getElementById("year").value || "",
    month: document.getElementById("month").value || "",
    start: document.getElementById("start").value || "",
    end: document.getElementById("end").value || ""
  };
  const res = await fetch("/summary?"+ new URLSearchParams(filters));
  const text = await res.text();
  document.getElementById("aiSummary").innerText = text;
}

loadYears();
applyFilters();
</script>
</body></html>
"""
(TEMPLATE_DIR / "dashboard.html").write_text(dashboard_html)

# ----------------- Routes -----------------
@app.route("/")
def landing_page():
    return render_template("landing.html")

@app.route("/login", methods=["GET","POST"])
def login():
    if request.method == "POST":
        email = request.form.get("email")
        pwd = request.form.get("password")
        user = User.query.filter_by(email=email).first()
        if user and bcrypt.check_password_hash(user.password, pwd):
            session["user_id"] = user.id
            session["session_id"] = str(uuid.uuid4())
            return redirect("/upload")
        else:
            return render_template("login.html", error="Invalid credentials")
    return render_template("login.html")

@app.route("/register", methods=["GET","POST"])
def register():
    if request.method == "POST":
        username = request.form.get("username")
        email = request.form.get("email")
        pwd = request.form.get("password")
        if User.query.filter_by(email=email).first():
            return render_template("register.html", error="Email already used")
        hashed = bcrypt.generate_password_hash(pwd).decode("utf-8")
        u = User(username=username, email=email, password=hashed)
        db.session.add(u); db.session.commit()
        return redirect("/login")
    return render_template("register.html")

@app.route("/logout")
def logout():
    sid = session.get("session_id")
    if sid:
        d = UPLOAD_ROOT / sid
        if d.exists(): shutil.rmtree(d, ignore_errors=True)
    session.clear()
    return redirect("/")

@app.before_request
def require_login():
    path = request.path
    protected = ["/dashboard","/upload","/analysis","/summary","/years","/load_dashboard"]
    # Landing, login, register are public
    if any(path.startswith(p) for p in protected):
        if "user_id" not in session:
            return redirect("/login")

@app.route("/upload", methods=["GET","POST"])
def upload():
    preview_html = None
    message = None
    if request.method == "POST":
        f = request.files.get("file")
        if f and f.filename.lower().endswith(".csv"):
            try:
                path = save_uploaded_csv(f)
                dfp = pd.read_csv(path, nrows=10)
                preview_html = dfp.to_html(classes="preview", index=False)
            except Exception as e:
                message = f"Preview failed: {e}"
        else:
            message = "Please upload a CSV file."
    return render_template("upload.html", preview=preview_html, message=message)

@app.route("/load_dashboard", methods=["POST"])
def load_dashboard():
    p = get_user_csv_path()
    if not p:
        return render_template("upload.html", preview=None, message="No CSV available. Upload a CSV or ensure default file exists.")
    return redirect("/dashboard")

@app.route("/dashboard")
def dashboard():
    return render_template("dashboard.html")

@app.route("/years")
def years_api():
    path = get_user_csv_path()
    if not path:
        return jsonify([])
    try:
        df, meta = load_and_prepare(path)
    except Exception:
        return jsonify([])
    yrs = sorted(df["year"].dropna().unique().tolist())
    return jsonify(yrs)

@app.route("/analysis")
def analysis_api():
    path = get_user_csv_path()
    if not path:
        return jsonify({"monthly": [], "category": [], "peak": [{"day":"No Data","amount":0}], "yearly": [], "summary": {}})
    try:
        df, meta = load_and_prepare(path)
    except Exception as e:
        return jsonify({"monthly": [], "category": [], "peak": [{"day":"No Data","amount":0}], "yearly": [], "summary": {}, "error": str(e)})

    year = request.args.get("year")
    month = request.args.get("month")
    start = request.args.get("start")
    end = request.args.get("end")

    temp = df.copy()
    if year:
        try: temp = temp[temp["year"] == int(year)]
        except: pass
    if month:
        try: temp = temp[temp["month"] == int(month)]
        except: pass
    if start:
        try: temp = temp[temp[meta["date_col"]] >= pd.to_datetime(start)]
        except: pass
    if end:
        try: temp = temp[temp[meta["date_col"]] <= pd.to_datetime(end)]
        except: pass

    expenses = temp[temp["expense"] > 0].copy()

    monthly = expenses.groupby("month_year")["expense"].sum().reset_index().rename(columns={"expense":"amount"})
    category = expenses.groupby("category_clean")["expense"].sum().reset_index().rename(columns={"category_clean":"category","expense":"amount"}).sort_values("amount", ascending=False)
    peak_data = expenses.groupby("day")["expense"].sum()
    if len(peak_data) > 0:
        peak = peak_data.reset_index(name="amount").sort_values("amount", ascending=False).head(10)
    else:
        peak = pd.DataFrame({"day":["No Data"], "amount":[0]})
    yearly = expenses.groupby("year")["expense"].sum().reset_index().rename(columns={"expense":"amount"})

    total = float(expenses["expense"].sum()) if len(expenses)>0 else 0.0
    top_cat = category.iloc[0]["category"] if len(category)>0 else None
    peak_row = peak.iloc[0] if len(peak)>0 else None
    summary = {
        "total": total,
        "top_cat": top_cat,
        "peak_day": str(peak_row["day"]) if peak_row is not None else None,
        "peak_amount": float(peak_row["amount"]) if peak_row is not None else 0.0
    }

    return jsonify({
        "monthly": monthly.to_dict(orient="records"),
        "category": category.to_dict(orient="records"),
        "peak": peak.to_dict(orient="records"),
        "yearly": yearly.to_dict(orient="records"),
        "summary": summary
    })

@app.route("/summary")
def summary_api():
    path = get_user_csv_path()
    if not path:
        return "No dataset uploaded."
    try:
        df, meta = load_and_prepare(path)
    except Exception as e:
        return "Failed to load dataset: " + str(e)

    year = request.args.get("year")
    month = request.args.get("month")
    start = request.args.get("start")
    end = request.args.get("end")
    temp = df.copy()
    if year:
        try: temp = temp[temp["year"] == int(year)]
        except: pass
    if month:
        try: temp = temp[temp["month"] == int(month)]
        except: pass
    if start:
        try: temp = temp[temp[meta["date_col"]] >= pd.to_datetime(start)]
        except: pass
    if end:
        try: temp = temp[temp[meta["date_col"]] <= pd.to_datetime(end)]
        except: pass

    expenses = temp[temp["expense"] > 0]
    if len(expenses)==0:
        return "No data available for the selected period."
    total = float(expenses["expense"].sum())
    cat_sums = expenses.groupby("category_clean")["expense"].sum().sort_values(ascending=False)
    top_cat = cat_sums.index[0]
    top_amt = cat_sums.iloc[0]
    insight_lines = [
        f"🧠 AI Summary (filtered range)",
        f"• Total spending: ₹{total:,.2f}",
        f"• Top category: {top_cat} (₹{top_amt:,.2f})",
    ]
    try:
        last_month = expenses["month_year"].max()
        prev = expenses[expenses["month_year"] < last_month].groupby("month_year")["expense"].sum()
        last_val = expenses[expenses["month_year"]==last_month]["expense"].sum()
        prev_mean = prev.mean() if len(prev)>0 else 0
        if prev_mean>0:
            pct = (last_val - prev_mean)/prev_mean*100
            if pct > 10:
                insight_lines.append(f"• Spending increased ~{pct:.0f}% vs previous months — review {top_cat}.")
            elif pct < -10:
                insight_lines.append("• Spending decreased compared with previous months — good job!")
    except Exception:
        pass
    insight_lines.append("• Suggestion: set a limit for the top category to reduce recurring spend.")
    return "\n".join(insight_lines)

# ----------------- Start ngrok and run -----------------
ngrok.set_auth_token(NGROK_TOKEN)
try: ngrok.kill()
except: pass
public_url = ngrok.connect(5000).public_url
print("🌍 PUBLIC URL:", public_url)
print("Open the URL, register/login, upload a CSV (or use default), then 'Load & Continue to Dashboard'.")
app.run(port=5000)

🌍 PUBLIC URL: https://waggly-bristol-swirly.ngrok-free.dev
Open the URL, register/login, upload a CSV (or use default), then 'Load & Continue to Dashboard'.
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [21/Nov/2025 12:58:47] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Nov/2025 12:59:26] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Nov/2025 12:59:26] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Nov/2025 12:59:39] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Nov/2025 12:59:41] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [21/Nov/2025 13:00:00] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Nov/2025 13:00:02] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [21/Nov/2025 13:00:06] "GET /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Nov/2025 13:00:07] "GET /static/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Nov/2025 13:05:08] "GET / HTTP/1.1" 200 -
INFO:werkzeug